In [1]:
import numpy as np

In [2]:
import FortranERI as FE

In [3]:
print(FE.lol.__doc__)

eri = runeri(basisidx,basisfloat,basisint,max_angular_moment,e1arr,e2arr,e3arr)

Wrapper for ``runeri``.

Parameters
----------
basisidx : input rank-2 array('i') with bounds (f2py_basisidx_d0,f2py_basisidx_d1)
basisfloat : input rank-2 array('d') with bounds (f2py_basisfloat_d0,f2py_basisfloat_d1)
basisint : input rank-2 array('i') with bounds (f2py_basisint_d0,f2py_basisint_d1)
max_angular_moment : input int
e1arr : input rank-5 array('d') with bounds (f2py_e1arr_d0,f2py_e1arr_d1,f2py_e1arr_d2,f2py_e1arr_d3,f2py_e1arr_d4)
e2arr : input rank-5 array('d') with bounds (f2py_e2arr_d0,f2py_e2arr_d1,f2py_e2arr_d2,f2py_e2arr_d3,f2py_e2arr_d4)
e3arr : input rank-5 array('d') with bounds (f2py_e3arr_d0,f2py_e3arr_d1,f2py_e3arr_d2,f2py_e3arr_d3,f2py_e3arr_d4)

Returns
-------
eri : rank-4 array('d') with bounds (size(basisidx,1),size(basisidx,1),size(basisidx,1),size(basisidx,1))
r1 = r(l1l2,m1m2,n1n2,cx,cy,cz,px,py,pz,p,rbuffer)

Wrapper for ``r``.

Parameters
----------
l1l2 : input int
m1m2

In [4]:
from scipy.special import hyp1f1

def R(l1l2, m1m2, n1n2, Cx, Cy, Cz, Px,  Py, Pz, p, R1, Rbuffer, check=0):
    # check = 0, normal calculation. 
    # check = 1, derivative calculation
    
    PCx = Px-Cx
    PCy = Py-Cy
    PCz = Pz-Cz
    RPC = ((PCx)**2+(PCy)**2+(PCz)**2)**0.5
    if check == 0:
        for t in range(0, l1l2+1):
            for u in range(0, m1m2+1):
                for v in range(0, n1n2+1):
                    # Check the range of n, to ensure no redundent n are calculated
                    if t == u == 0:
                        exclude_from_n = v
                    elif t == 0:
                        exclude_from_n = n1n2 + u
                    else:
                        exclude_from_n = n1n2 + m1m2 + t
                    for n in range(0, l1l2+m1m2+n1n2+1-exclude_from_n):
                        val = 0.0
                        if t == u == v == 0:
                            Rbuffer[t,u,v,n] = (-2.0*p)**n*boys(n,p*RPC*RPC)
                        else:
                            if t == u == 0:
                                if v > 1:
                                    val += (v-1)*Rbuffer[t,u,v-2,n+1]
                                val += PCz*Rbuffer[t,u,v-1,n+1]  
                            elif t == 0:
                                if u > 1:
                                    val += (u-1)*Rbuffer[t,u-2,v,n+1]
                                val += PCy*Rbuffer[t,u-1,v,n+1]
                            else:
                                if t > 1:
                                    val += (t-1)*Rbuffer[t-2,u,v,n+1]
                                val += PCx*Rbuffer[t-1,u,v,n+1]
                            Rbuffer[t,u,v,n] = val
                            
                        if n == 0:
                            R1[t,u,v] = Rbuffer[t,u,v,n]

    return R1

def boys(m,T):
    return hyp1f1(m+0.5,m+1.5,-T)/(2.0*m+1.0) 

def elelrep(p,  q,  l1,  l2,  l3,  l4,  m1,  m2,  m3, m4, n1, n2, n3, n4, N1,  N2,  N3,  N4,  c1,  c2,  c3,  c4, E1, E2, E3, E4, E5, E6, Rpre):

    N = N1*N2*N3*N4*c1*c2*c3*c4
    pi = 3.141592653589793238462643383279
    
    val = 0.0
    for tau in range(l3+l4+1):
        for nu in range(m3+m4+1):
            for phi in range(n3+n4+1):
                factor = (-1.0)**(tau+nu+phi)*E4[tau]*E5[nu]*E6[phi]
                for t in range(l1+l2+1):
                    for u in range(m1+m2+1):
                        for v in range(n1+n2+1):
                            val += E1[t]*E2[u]*E3[v]*Rpre[t+tau,u+nu,v+phi]*factor

    val *= 2.0*pi**2.5/(p*q*(p+q)**0.5)
    print(val)
    return val*N

def runCythonIntegrals(basisidx, basisfloat, basisint, E1arr, E2arr, E3arr):
    
    ERI = np.zeros((len(basisidx),len(basisidx),len(basisidx),len(basisidx)))
    R1buffer = np.zeros((4*np.max(basisint)+1,4*np.max(basisint)+1,4*np.max(basisint)+1))
    Rbuffer = np.zeros((4*np.max(basisint)+1,4*np.max(basisint)+1,4*np.max(basisint)+1,3*4*np.max(basisint)+1))
    # basisidx [number of primitives, start index in basisfloat and basisint]
    # basisfloat array of float values for basis, N, zeta, c, x, y, z 
    # basisint array of integer values for basis, l, m, n, atomidx
    
    # Run ERI
    for mu in range(0, len(basisidx)):
        for nu in range(mu, len(basisidx)):
            munu = mu*(mu+1)//2+nu
            for lam in range(0, len(basisidx)):
                for sig in range(lam, len(basisidx)):
                    lamsig = lam*(lam+1)//2+sig
                    if munu >= lamsig:
                        calc = 0.0
                        for i in range(basisidx[mu,1],basisidx[mu,1]+basisidx[mu,0]):
                            N1 = basisfloat[i,0]
                            a  = basisfloat[i,1]
                            c1 = basisfloat[i,2]
                            Ax = basisfloat[i,3]
                            Ay = basisfloat[i,4]
                            Az = basisfloat[i,5]
                            l1 = basisint[i,0]
                            m1 = basisint[i,1]
                            n1 = basisint[i,2]
                            for j in range(basisidx[nu,1],basisidx[nu,1]+basisidx[nu,0]):
                                N2 = basisfloat[j,0]
                                b  = basisfloat[j,1]
                                c2 = basisfloat[j,2]
                                Bx = basisfloat[j,3]
                                By = basisfloat[j,4]
                                Bz = basisfloat[j,5]
                                l2 = basisint[j,0]
                                m2 = basisint[j,1]
                                n2 = basisint[j,2]

                                p   = a+b
                                Px  = (a*Ax+b*Bx)/p
                                Py  = (a*Ay+b*By)/p
                                Pz  = (a*Az+b*Bz)/p
                                
                                E1 = E1arr[mu,nu,i-basisidx[mu,1],j-basisidx[nu,1]]
                                E2 = E2arr[mu,nu,i-basisidx[mu,1],j-basisidx[nu,1]]
                                E3 = E3arr[mu,nu,i-basisidx[mu,1],j-basisidx[nu,1]]
                                for k in range(basisidx[lam,1],basisidx[lam,1]+basisidx[lam,0]):
                                    N3 = basisfloat[k,0]
                                    c  = basisfloat[k,1]
                                    c3 = basisfloat[k,2]
                                    Cx = basisfloat[k,3]
                                    Cy = basisfloat[k,4]
                                    Cz = basisfloat[k,5]
                                    l3 = basisint[k,0]
                                    m3 = basisint[k,1]
                                    n3 = basisint[k,2]
                                    for l in range(basisidx[sig,1],basisidx[sig,1]+basisidx[sig,0]):
                                        N4 = basisfloat[l,0]
                                        d  = basisfloat[l,1]
                                        c4 = basisfloat[l,2]
                                        Dx = basisfloat[l,3]
                                        Dy = basisfloat[l,4]
                                        Dz = basisfloat[l,5]
                                        l4 = basisint[l,0]
                                        m4 = basisint[l,1]
                                        n4 = basisint[l,2]
                                                                                    
                                        q   = c+d
                                        Qx  = (c*Cx+d*Dx)/q
                                        Qy  = (c*Cy+d*Dy)/q
                                        Qz  = (c*Cz+d*Dz)/q

                                        E4 = E1arr[lam,sig,k-basisidx[lam,1],l-basisidx[sig,1]]
                                        E5 = E2arr[lam,sig,k-basisidx[lam,1],l-basisidx[sig,1]]
                                        E6 = E3arr[lam,sig,k-basisidx[lam,1],l-basisidx[sig,1]]
                                        
                                        alpha = p*q/(p+q)
                                        
                                        R1 = R(l1+l2+l3+l4, m1+m2+m3+m4, n1+n2+n3+n4, Qx, Qy, Qz, Px, Py, Pz, alpha, R1buffer, Rbuffer)
                                        calc += elelrep(p,q,l1, l2, l3, l4, m1, m2, m3, m4, n1, n2, n3, n4, N1, N2, N3, N4, c1, c2, c3, c4, E1, E2, E3, E4, E5, E6, R1)
                                        
                        ERI[mu,nu,lam,sig] = ERI[nu,mu,lam,sig] = ERI[mu,nu,sig,lam] = ERI[nu,mu,sig,lam] = ERI[lam,sig,mu,nu] = ERI[sig,lam,mu,nu] = ERI[lam,sig,nu,mu] = ERI[sig,lam,nu,mu] = calc            
                                
    #END OF run ERI
    return ERI

In [5]:
import numpy as np
basisidx           = np.load('basisidx.npy')
basisfloat         = np.load('basisfloat.npy')
basisint           = np.load('basisint.npy')
max_angular_moment = np.max(basisint)
E1arr              = np.load('E1arr.npy')
E2arr              = np.load('E2arr.npy')
E3arr              = np.load('E3arr.npy')

In [6]:
A = FE.lol.runeri(basisidx,basisfloat,basisint,max_angular_moment,E1arr,E2arr,E3arr)

In [7]:
B = runCythonIntegrals(basisidx, basisfloat, basisint, E1arr, E2arr, E3arr)

0.201411245209
0.382041244885
0.439688692732
0.382041244885
1.43823560882
2.33021467535
0.439688692732
2.33021467535
5.64062053888
0.382041244885
0.749852925973
0.869578303123
0.749852925973
3.00863872101
4.95316834559
0.869578303123
4.95316834559
12.2156516756
0.439688692732
0.869578303123
1.01021167233
0.869578303123
3.54527393323
5.86258801963
1.01021167233
5.86258801963
14.5369117756
0.382041244885
0.749852925973
0.869578303123
0.749852925973
3.00863872101
4.95316834559
0.869578303123
4.95316834559
12.2156516756
1.43823560882
3.00863872101
3.54527393323
3.00863872101
14.2233970314
24.7585285386
3.54527393323
24.7585285386
65.9351238159
2.33021467535
4.95316834559
5.86258801963
4.95316834559
24.7585285386
44.2102380154
5.86258801963
44.2102380154
122.908401073
0.439688692732
0.869578303123
1.01021167233
0.869578303123
3.54527393323
5.86258801963
1.01021167233
5.86258801963
14.5369117756
2.33021467535
4.95316834559
5.86258801963
4.95316834559
24.7585285386
44.2102380154
5.86258801963

0.0772805740732
0.0810925584176
0.248006656351
0.458663654966
0.540307140667
1.90182910463e-10
2.04418062779e-10
2.07508316528e-10
9.10941675809e-09
1.30342475718e-08
1.41186642368e-08
8.91506131297e-08
2.40064719026e-07
3.10931395636e-07
4.58425300855e-08
4.92741930145e-08
5.00191654443e-08
2.20285306853e-06
3.15595852879e-06
3.41970092752e-06
2.20488169401e-05
6.14721624301e-05
8.07799979729e-05
5.09851954338e-07
5.48018277571e-07
5.56303724058e-07
2.45001496777e-05
3.51008948101e-05
3.80343571402e-05
0.0002452849403
0.000684310443749
0.000899589049529
4.4308608867e-08
4.76186353806e-08
4.83370929928e-08
2.0462715725e-06
2.89835805181e-06
3.13147319579e-06
1.7911437682e-05
4.39461870887e-05
5.52734526382e-05
6.50701882934e-07
6.99404385157e-07
7.09976985401e-07
3.11574555717e-05
4.45864375461e-05
4.8297823746e-05
0.000306753284721
0.000837832944668
0.00109223974479
3.32422412102e-06
3.57306008976e-06
3.62707923849e-06
0.000159622325454
0.000228630280295
0.00024772099361
0.00159198836

0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
2.18036205313e-17
6.68432180613e-12
3.15936060317e-10
6.68432180613e-12
5.25992545728e-10
4.80351405691e-09
3.15936060317e-10
4.80351405691e-09
2.37148548949e-08
6.68432180613e-12
9.01332846579e-06
0.000496925768988
2.47688928675e-06
0.000259078005948
0.00409124859422
0.000131529745795
0.0021285605381
0.0150800390809
3.15936060317e-10
0.000496925768988
0.0319338110216
0.000131529745795
0.0131505842716
0.235223634839
0.00711238623942
0.112914718012
0.849073682636
6.68432180613e-12
2.47688928675e-06
0.000131529745795
9.01332846579e-06
0.000259078005948
0.0021285605381
0.000496925768988
0.00409124859422
0.0150800390809
5.25992545728e-10
0.000259078005948
0.0131505842716
0.000259078005948
0.0175734705224
0.182656441654
0.0131505842716
0.182656441654
0.936515705799
4.80351405691e-09
0.00409124859422
0.235223634839
0.0021285605381
0.182656441654
2.55085167623
0.112914718012
1.72980338798
11.1470246169
3.15936060317e-10
0.000131529745795
0.0071123862394

0.00661972228872
0.00793369786594
0.00825803439966
0.0202134266
0.0322268849104
0.0363973615695
0.000842556856303
0.000877909486018
0.000885436395547
0.00688637604004
0.00825803439966
0.00859673762106
0.0211025884167
0.0336887164954
0.0380600896484
0.00185091880927
0.00193064567996
0.00194763736235
0.0166458580729
0.0202134266
0.0211025884167
0.0554423292722
0.0916463869441
0.104426812123
0.00282757406654
0.00295044131114
0.00297663645585
0.0263895380338
0.0322268849104
0.0336887164954
0.0916463869441
0.154693104304
0.177237995291
0.0031633072986
0.00330101631147
0.00333037790597
0.0297654883534
0.0363973615695
0.0380600896484
0.104426812123
0.177237995291
0.203372657546
7.34506707919e-07
1.61633398705e-06
1.9328245727e-06
1.61633398705e-06
9.32010480394e-06
1.71994338047e-05
1.9328245727e-06
1.71994338047e-05
3.98937140643e-05
7.89490162317e-07
1.73732896437e-06
2.07751127185e-06
1.73732896437e-06
1.00176053688e-05
1.84861403058e-05
2.07751127185e-06
1.84861403058e-05
4.28776402854e-0

0.0
0.0
5.47092299834e-22
-1.12936565363e-21
0.0
0.0
0.0
-1.38929851471e-21
0.0
0.0
0.0
5.75575618061e-22
-1.20461881857e-21
0.0
0.0
0.0
-1.48981992294e-21
0.0
2.77429735359e-21
2.88480843393e-21
3.48994368886e-21
1.78675250634e-20
2.24725953136e-20
2.32996632239e-20
5.26072251401e-20
7.95950595986e-20
9.09119628136e-20
-1.40186232383e-20
-1.4629996554e-20
-9.21144796251e-21
-1.53700763112e-19
-1.55310092115e-19
-1.62074357537e-19
-4.18177732947e-19
-7.32456818359e-19
-7.73468490615e-19
0.0
0.0
6.64655436797e-21
-3.28206237555e-20
0.0
0.0
0.0
-6.96479440195e-20
0.0
0.0
0.0
6.9157986709e-21
-3.46508036053e-20
0.0
0.0
0.0
-7.46139961231e-20
0.0
0.0
0.0
1.65686944184e-20
-1.1016748466e-19
0.0
0.0
0.0
-3.38147913768e-19
0.0
-5.37099290564e-20
-5.60839511968e-20
-3.05626180423e-20
-7.32456818359e-19
-6.65965104676e-19
-6.97980474844e-19
-2.03394672892e-18
-4.29359615372e-18
-4.14427131685e-18
0.0
0.0
2.92875922817e-20
-2.22696315462e-19
0.0
0.0
0.0
-8.64709596671e-19
0.0
0.0
0.0
0.0
0.0
0.0

...

0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
2.11988876046e-11
1.17990823495e-05
0.000633769153375
1.17990823495e-05
0.000876404645468
0.00962984732591
0.000633769153375
0.00962984732591
0.0537323889296
6.97195488956e-11
3.91591838303e-05
0.00210902202301
3.91591838303e-05
0.00287596971899
0.0318408833061
0.00210902202301
0.0318408833061
0.178026488719
9.72204357495e-11
5.47992957435e-05
0.00295443589643
5.47992957435e-05
0.00400813694128
0.0444995215128
0.00295443589643
0.0444995215128
0.248996422804
6.97195488956e-11
3.91591838303e-05
0.00210902202301
3.91591838303e-05
0.00287596971899
0.0318408833061
0.00210902202301
0.0318408833061
0.178026488719
7.30054867468e-10
0.000427045494564
0.0232861134384
0.000427045494564
0.0301203618458
0.34323539746
0.0232861134384
0.34323539746
1.93607214007
1.90310174399e-09
0.00114674336019
0.0631781539678
0.00114674336019
0.0789441957775
0.917161866694
0.0631781539678
0.917161866694
5.20912180177
9.72204357495e-11


1.10384525222
0.00727167248669
0.023744261711
0.0330141910402
0.023744261711
0.241099706941
0.616840724374
0.0330141910402
0.616840724374
2.94147013291
0.000533126071758
0.00163063737694
0.00221809789556
0.00163063737694
0.013885097097
0.0330141910402
0.00221809789556
0.0330141910402
0.142834327276
0.00727167248669
0.023744261711
0.0330141910402
0.023744261711
0.241099706941
0.616840724374
0.0330141910402
0.616840724374
2.94147013291
0.0308085419906
0.102129764794
0.142834327276
0.102129764794
1.10384525222
2.94147013291
0.142834327276
2.94147013291
15.1700195214
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.55798694437e-07
6.73541678349e-05
0.000123486159488
6.5263179292e-05
0.00126773399273
0.0022019691695
0.

0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
2.64876305592e-08
8.13070991028e-08
1.101015379e-07
8.13070991028e-08
6.05425458127e-07
1.24170033118e-06
1.101015379e-07
1.24170033118e-06
3.63209650529e-06
0.000133265854

7.30192276324e-09
3.41706334014e-07
3.80036328238e-05
0.00094992689031
0.0124477374953
0.0929147596065
0.00674673388139
0.0862027883093
0.598707573586
2.42147808448e-08
1.14148391024e-06
0.000129508742849
0.00314552728277
0.0412370420752
0.308439540687
0.0223354267408
0.285194262793
1.97729113941
4.34072046764e-06
0.00141016777454
0.0170329411319
0.00544139526828
0.0525751066371
0.229175019247
0.0343260353742
0.273296977341
1.08435743664
9.55206433142e-06
0.00310317459003
0.0374821997475
0.0119741412834
0.115680336852
0.50418351555
0.0755363280457
0.600825959348
2.37838663245
1.14224429408e-05
0.00371080371957
0.0448215473291
0.0143187314926
0.138322864231
0.602844958962
0.0903263103944
0.718203861677
2.8410935531
9.55206433142e-06
0.00310317459003
0.0374821997475
0.0119741412834
0.115680336852
0.50418351555
0.0755363280457
0.600825959348
2.37838663245
5.58163434055e-05
0.0181383791522
0.219095239537
0.0698434686765
0.67350062814
2.93600309047
0.440246632447
3.46981826617
13.6137914023

0.357870171508
0.502054493907
0.357870171508
4.01701028729
11.0481423234
0.502054493907
11.0481423234
61.9355549169
-2.16461836559e-05
-0.000117860324581
-0.000189760373291
-0.000117860324581
-0.00354242772442
-0.014310912239
-0.000189760373291
-0.014310912239
-0.125065436952
-4.76272610761e-05
-0.000259173965675
-0.000417098717699
-0.000259173965675
-0.00767625980431
-0.0304384416379
-0.000417098717699
-0.0304384416379
-0.259232498481
-5.69410012722e-05
-0.000309707834712
-0.000498282264202
-0.000309707834712
-0.00911925169789
-0.0359623905337
-0.000498282264202
-0.0359623905337
-0.304247994625
-4.76272610761e-05
-0.000259173965675
-0.000417098717699
-0.000259173965675
-0.00767625980431
-0.0304384416379
-0.000417098717699
-0.0304384416379
-0.259232498481
-0.00025405567379
-0.00134941029653
-0.00215096617401
-0.00134941029653
-0.0359846972799
-0.134360789262
-0.00215096617401
-0.134360789262
-1.08555252274
-0.000399437493637
-0.00209870909171
-0.00333316515045
-0.00209870909171
-0.0543

In [8]:
np.max(np.abs(A-B))

3.3289145573434098e-07

In [9]:
np.abs(A-B)

array([[[[  5.38884350e-08,   1.29683866e-09,   1.63779624e-08, ...,
            1.12342515e-08,   0.00000000e+00,   6.35537126e-09],
         [  1.29683866e-09,   3.27476757e-08,   7.73548506e-09, ...,
            4.41099045e-10,   0.00000000e+00,   1.24465125e-09],
         [  1.63779624e-08,   7.73548506e-09,   3.16302153e-08, ...,
            4.82928909e-09,   0.00000000e+00,   1.05692172e-08],
         ..., 
         [  1.12342515e-08,   4.41099045e-10,   4.82928909e-09, ...,
            3.17860661e-08,   0.00000000e+00,   6.73093656e-09],
         [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00, ...,
            0.00000000e+00,   3.03842059e-08,   0.00000000e+00],
         [  6.35537126e-09,   1.24465125e-09,   1.05692172e-08, ...,
            6.73093656e-09,   0.00000000e+00,   2.10472110e-08]],

        [[  1.29683866e-09,   2.56229548e-10,   8.43286869e-10, ...,
            4.06976798e-10,   0.00000000e+00,   2.84659512e-10],
         [  2.56229548e-10,   8.47246562e-09,